## Importing libraries

In [17]:
# Giving a seed value.Apparently you may use different seed values at each stage
seed_value= 0

# 4 steps that are neeeded to be done for the code to be reproducible.

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

# 5. Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))

#Import all the necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import time
from matplotlib import pyplot

#Access all the necessary variables and datasets
#appended_data has all the chemistry compositions and microalloy elements compositions and output column been modified.
%store -r appended_data
%store -r dict_const
%store -r dict_normal
%store -r dict_uniform

#Creating a output column
df=appended_data.copy()
output_columns = ['NF1', 'NF2', 'NF1-T', 'NF1-B', 'NF2-T', 'NF2-B']
df['FINALOUTPUT'] = df[output_columns].apply(lambda row: 0 if 'Defect' in row.values else 1, axis=1)
df.head(3)

GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


,DATE,TIME,HEATNO,GRADE,GRADECATEGORY,SPEED,STOPPER,TUNDISHWEIGHT,TUNDISHTEMPERATURE,BROADFACE1WATERFLOW,...,PER_MNS,PER_VC,PER_NBN,NF1,NF1-T,NF2,NF2-T,NF1-B,NF2-B,FINALOUTPUT
0,2022-02-09 00:00:00,00:49:43,P16391,SRLNC22,LCMA,1.120,49,23.83,1555.0,3824,...,0.002926,0.000371,0.000083,No Defect,No Defect,No Defect,No Defect,No Defect,No Defect,1
1,2022-02-09 00:00:00,00:49:46,P16391,SRLNC22,LCMA,1.125,50,23.79,1555.0,3824,...,0.002926,0.000371,0.000083,No Defect,No Defect,No Defect,No Defect,No Defect,No Defect,1
2,2022-02-09 00:00:00,00:49:49,P16391,SRLNC22,LCMA,1.122,50,23.76,1555.0,3824,...,0.002926,0.000371,0.000083,No Defect,No Defect,No Defect,No Defect,No Defect,No Defect,1


In [20]:
x_axis=df.iloc[:,0:114]

uniqueheats=df['HEATNO'].unique()
rows,cols=(len(uniqueheats),2)
print(len(uniqueheats)) #591 unique heats

print(x_axis.shape)
print(x_axis.columns)
x_axis.head(3)

# The new added variables which include compositions and microalloy compositions fall under the constant distribution.
compositions_variables=x_axis.columns[86:115]
print(compositions_variables)

591
(97425, 114)
Index(['DATE', 'TIME', 'HEATNO', 'GRADE', 'GRADECATEGORY', 'SPEED', 'STOPPER',
       'TUNDISHWEIGHT', 'TUNDISHTEMPERATURE', 'BROADFACE1WATERFLOW',
       ...
       'DAYSAFTERSPRAYCLEANING', 'PER_ALC', 'PER_ALN', 'PER_NBC', 'PER_VN',
       'PER_TIN', 'PER_TIC', 'PER_MNS', 'PER_VC', 'PER_NBN'],
      dtype='object', length=114)
Index(['PER_AL', 'PER_C', 'PER_CA', 'PER_NB', 'PER_CR', 'PER_CU', 'PER_MN',
       'PER_MO', 'PER_N', 'PER_NI', 'PER_P', 'PER_S', 'PER_SI', 'PER_TI',
       'PER_V', 'DAYSAFTERSHUTDOWN', 'DAYSAFTERNARROWFACEALIGNMENT',
       'DAYSAFTERCIRCLARD', 'DAYSAFTERSPRAYCLEANING', 'PER_ALC', 'PER_ALN',
       'PER_NBC', 'PER_VN', 'PER_TIN', 'PER_TIC', 'PER_MNS', 'PER_VC',
       'PER_NBN'],
      dtype='object')


### Creating a multidimensional list

#### Elements of a list within a list are: 1) Heat No 2) Dataframe of all parameters values corresponding to a particular heat number
#### 3) Grade No 4) Binary value indictaing defective or not defective item.

In [28]:
new_data=[[0,0,0,0] for i in range(rows)] # multidimensional-list

i=0
for heat_value in uniqueheats:
    new_data[i][0]=heat_value
    
    data_range= df.loc[df['HEATNO'] == heat_value] #accessing the range of each unique heat number
    
    uniquegrade=data_range['GRADE'].unique()
    
    new_data[i][1]=data_range
    new_data[i][2]=uniquegrade[0] # to access uniquegrade element
    new_data[i][3]=data_range['FINALOUTPUT'].unique()[0]
    i=i+1

new_data[1][1] #accessing grade
new_data[1][1]['GRADECATEGORY'].unique()[0] #accessing gradde category

'LCMA'

### This section of code creates a dataframe which has the statistical features of the parameters and grade category.

In [31]:
# The function variable_constant is going to calcluate mean statistical feature for the variable passed to it as argument for each heat number.
# The function returns a dataframe named Grade_data after appending mean values of all 591 heats.

def variable_constant(variable):
    for row in range (0,len(new_data)): #will walk through all heat numbers
        mean_value=np.mean(new_data[row][1][variable])
        mean_array.append(mean_value)
    Grade_data[variable+'MEAN']=mean_array

# The function variable_uniform is going to calcluate statistical features mean,minimum and maximum for the variable passed to it as argument for each heat number.
# The function returns a dataframe named Grade_data after appending all the statistical features for all 591 heats.

def variable_uniform(variable):
    for row in range (0,len(new_data)): #will walk through all heat numbers
        mean_value=np.mean(new_data[row][1][variable])
        max_value=np.max(new_data[row][1][variable])
        min_value=np.min(new_data[row][1][variable])
        mean_array.append(mean_value)
        min_array.append(min_value)
        max_array.append(max_value)                   
    Grade_data[variable+'MEAN']=mean_array
    Grade_data[variable+'MIN']=min_array
    Grade_data[variable+'MAX']=max_array

# The function variable_normal is going to calcluate statistical features mean and variance for the variable passed to it as argument for each heat number.
# The function returns a dataframe named Grade_data after appending all the statistical features for all 591 heats.

def variable_normal(variable):
    for row in range (0,len(new_data)): #will walk through all heat numbers
        mean_value=np.mean(new_data[row][1][variable])
        var_value=np.var(new_data[row][1][variable])
        mean_array.append(mean_value)
        var_array.append(var_value)
    Grade_data[variable+'MEAN']=mean_array
    Grade_data[variable+'VARIANCE']=var_array                      

# The loop walks through all the independent variables. 
# Places a condition in which dictionary the variable lies and calls the respective functions.

const_selected=[]
uniform_selected=[]
normal_selected=[]
Grade_data = pd.DataFrame()
for variable_index in range(0,len(x_axis.columns),1):
    #print(x_axis.columns[variable_index])
    if x_axis.columns[variable_index] in dict_const:
        mean_array=[]
        current_variable=x_axis.columns[variable_index]
        variable_constant(current_variable)
        const_selected.append(x_axis.columns[variable_index])
    if x_axis.columns[variable_index] in compositions_variables:
        mean_array=[]
        current_variable=x_axis.columns[variable_index]
        variable_constant(current_variable)
        const_selected.append(x_axis.columns[variable_index])
    if x_axis.columns[variable_index] in dict_uniform:
        mean_array=[]
        min_array=[]
        max_array=[]                   
        current_variable=x_axis.columns[variable_index]
        variable_uniform(current_variable)
        uniform_selected.append(x_axis.columns[variable_index])
    if x_axis.columns[variable_index] in dict_normal:
        mean_array=[]
        var_array=[]
        current_variable=x_axis.columns[variable_index]
        variable_normal(current_variable)
        normal_selected.append(x_axis.columns[variable_index])
op_data=[]
grade_category=[]
for row in range (0,len(new_data)):
    op_data.append(new_data[row][3])
    grade_category.append(new_data[row][1]['GRADECATEGORY'].unique()[0])
Grade_data["OUTPUT"]=op_data
Grade_data.insert(0,'GRADECATEGORY',grade_category)
print(len(const_selected))
print(len(uniform_selected))
print(len(normal_selected))
Grade_data

C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

64
12
31


C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Grade_data[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_8164\1037042809.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

,GRADECATEGORY,SPEEDMEAN,STOPPERMEAN,TUNDISHWEIGHTMEAN,TUNDISHTEMPERATUREMEAN,TUNDISHTEMPERATUREMIN,TUNDISHTEMPERATUREMAX,BROADFACE1WATERFLOWMEAN,BROADFACE1WATERFLOWVARIANCE,BROADFACE2WATERFLOWMEAN,...,PER_ALCMEAN,PER_ALNMEAN,PER_NBCMEAN,PER_VNMEAN,PER_TINMEAN,PER_TICMEAN,PER_MNSMEAN,PER_VCMEAN,PER_NBNMEAN,OUTPUT
0,LCMA,1.120270,50.758621,23.854885,1554.574713,1554.0,1555.0,3826.942529,5.341525,3820.425287,...,0.002079,0.000137,0.001267,0.000024,0.000043,0.000651,0.002926,0.000371,0.000083,1
1,LCMA,1.114253,56.146465,23.548081,1548.237374,1548.0,1549.0,3823.025253,9.519564,3823.176768,...,0.002678,0.000180,0.001382,0.000026,0.000053,0.000788,0.003059,0.000387,0.000093,1
2,LCMA,1.198736,61.086207,24.320115,1551.000000,1551.0,1551.0,3821.913793,6.366132,3821.626437,...,0.002037,0.000163,0.001260,0.000028,0.000048,0.000595,0.003990,0.000350,0.000101,0
3,LCMA,1.198899,65.455696,24.275696,1547.107595,1546.0,1548.0,3826.860759,8.828713,3826.151899,...,0.001939,0.000144,0.001344,0.000026,0.000048,0.000651,0.002660,0.000350,0.000100,0
4,LCMA,1.141729,66.005650,24.563785,1553.090395,1553.0,1554.0,3823.536723,9.514188,3823.525424,...,0.002113,0.000151,0.001285,0.000027,0.000043,0.000607,0.003725,0.000378,0.000092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,HC,0.983690,71.732143,23.541369,1545.142857,1545.0,1546.0,4000.803571,6.741178,4000.636905,...,0.006482,0.000170,0.000421,0.000013,0.000010,0.000366,0.003682,0.000513,0.000011,0
587,HCMA,0.795487,69.920635,27.779471,1555.682540,1554.0,1558.0,3958.724868,15125.331710,3966.460317,...,0.006992,0.000253,0.000426,0.000239,0.000004,0.000122,0.004050,0.006597,0.000015,1
588,HCMA,0.911294,73.304124,22.814948,1537.123711,1536.0,1538.0,4052.396907,28.796073,4024.128866,...,0.009100,0.000260,0.005642,0.000026,0.000081,0.002839,0.018383,0.000910,0.000161,1
589,HCMA,0.918199,67.461165,24.742087,1543.466019,1542.0,1545.0,4030.233010,223.101046,4027.310680,...,0.008918,0.000255,0.005460,0.000025,0.000076,0.002657,0.016292,0.000874,0.000156,1


## Select rows only with HCMA and HC grade category

In [33]:
Final_Grade_data = Grade_data[Grade_data['GRADECATEGORY'].isin(['HCMA', 'HC'])]
Final_Grade_data

,GRADECATEGORY,SPEEDMEAN,STOPPERMEAN,TUNDISHWEIGHTMEAN,TUNDISHTEMPERATUREMEAN,TUNDISHTEMPERATUREMIN,TUNDISHTEMPERATUREMAX,BROADFACE1WATERFLOWMEAN,BROADFACE1WATERFLOWVARIANCE,BROADFACE2WATERFLOWMEAN,...,PER_ALCMEAN,PER_ALNMEAN,PER_NBCMEAN,PER_VNMEAN,PER_TINMEAN,PER_TICMEAN,PER_MNSMEAN,PER_VCMEAN,PER_NBNMEAN,OUTPUT
9,HCMA,1.227896,48.083333,23.620278,1554.590278,1554.0,1556.0,4019.965278,65.755739,3999.236111,...,0.007500,0.000250,0.001935,0.000010,0.000003,0.000075,0.003419,0.000300,0.000064,0
10,HCMA,1.201590,51.231214,24.965434,1538.919075,1538.0,1539.0,4013.884393,14.171606,3993.855491,...,0.007723,0.000228,0.001806,0.000009,0.000002,0.000078,0.004400,0.000311,0.000053,1
11,HCMA,1.201949,53.464968,24.144395,1545.000000,1545.0,1545.0,4023.713376,3.223579,3998.114650,...,0.007595,0.000250,0.002263,0.000011,0.000003,0.000091,0.004110,0.000349,0.000075,1
12,HCMA,1.201726,51.529762,26.893869,1544.511905,1544.0,1545.0,4015.583333,9.731151,3991.047619,...,0.007138,0.000237,0.002214,0.000009,0.000003,0.000075,0.004256,0.000271,0.000073,1
13,HC,1.005774,48.208333,22.698333,1553.958333,1553.0,1554.0,3937.208333,18.283978,3912.428571,...,0.006290,0.000235,0.000342,0.000017,0.000009,0.000233,0.003291,0.000466,0.000013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,HC,0.983690,71.732143,23.541369,1545.142857,1545.0,1546.0,4000.803571,6.741178,4000.636905,...,0.006482,0.000170,0.000421,0.000013,0.000010,0.000366,0.003682,0.000513,0.000011,0
587,HCMA,0.795487,69.920635,27.779471,1555.682540,1554.0,1558.0,3958.724868,15125.331710,3966.460317,...,0.006992,0.000253,0.000426,0.000239,0.000004,0.000122,0.004050,0.006597,0.000015,1
588,HCMA,0.911294,73.304124,22.814948,1537.123711,1536.0,1538.0,4052.396907,28.796073,4024.128866,...,0.009100,0.000260,0.005642,0.000026,0.000081,0.002839,0.018383,0.000910,0.000161,1
589,HCMA,0.918199,67.461165,24.742087,1543.466019,1542.0,1545.0,4030.233010,223.101046,4027.310680,...,0.008918,0.000255,0.005460,0.000025,0.000076,0.002657,0.016292,0.000874,0.000156,1


In [43]:
total_features_list=Final_Grade_data.iloc[:,1:-1].columns.tolist()
total_features_list

['SPEEDMEAN',
 'STOPPERMEAN',
 'TUNDISHWEIGHTMEAN',
 'TUNDISHTEMPERATUREMEAN',
 'TUNDISHTEMPERATUREMIN',
 'TUNDISHTEMPERATUREMAX',
 'BROADFACE1WATERFLOWMEAN',
 'BROADFACE1WATERFLOWVARIANCE',
 'BROADFACE2WATERFLOWMEAN',
 'BROADFACE2WATERFLOWVARIANCE',
 'NARROWFACE1WATERFLOWMEAN',
 'NARROWFACE1WATERFLOWMIN',
 'NARROWFACE1WATERFLOWMAX',
 'NARROWFACE2WATERFLOWMEAN',
 'NARROWFACE2WATERFLOWMIN',
 'NARROWFACE2WATERFLOWMAX',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTAMEAN',
 'MOLDNARROWFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'MOLDNARROWFACE2INLETOUTLETWATERTEMP.DELTAMEAN',
 'MOLDBROADFACECOOLINGWATERINLETTEMPMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN',
 'MOLDBROADFACECOOLINGWATERINLETPRESSMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETPRESSMEAN',
 'BROADFACE1HEATFLUXMEAN',
 'BROADFACE1HEATFLUXMIN',
 'BROADFACE1HEATFLUXMAX',
 'BROADFACE2HEATFLUXMEAN',
 'BROADFACE2HEATFLUXMIN',
 'BROADFACE2HEATFLUXMAX',
 'NARROWFACE1HEATFLUXMEAN',
 'NARROWFACE1HEATFL

## Defining the function for training

In [44]:
def training_data(new_features):
    # Creating a new data
    new_data = Final_Grade_data.loc[:,new_features] # Taking the features given by forward selection algorithm.
    new_data = pd.concat([new_data, Final_Grade_data['OUTPUT']], axis=1)  # Add the 'OUTPUT' column
    new_data = shuffle(new_data, random_state=42) #Shuffling the data along with the output column.
    
    #Taking all independent variables from shuffled new_data in X
    X=new_data.iloc[:,:-1]

    # Splitting the shuffled new_data into train and test
    new_total_length = X.shape[0]
    new_train_size = int(X.shape[0] * 0.8)
    new_test_size = X.shape[0] - new_train_size

    new_train_X = X.iloc[:new_train_size, :].to_numpy().astype(np.float32)  # 2D array
    new_train_Y = new_data['OUTPUT'][:new_train_size].to_numpy().astype(np.float32)  # 1D array

    new_test_X = X.iloc[new_train_size:, :].to_numpy().astype(np.float32)  # 2D array
    new_test_Y = new_data['OUTPUT'][new_train_size:].to_numpy().astype(np.float32)  # 1D array

    # Scaling the data
    scaler = RobustScaler()
    new_train_X = scaler.fit_transform(new_train_X)
    new_test_X = scaler.transform(new_test_X)

    # Create a new model with the updated feature set (Random Forest Classifier)
    #new_model = RandomForestClassifier()
    new_model = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2,
                                       min_samples_leaf=1, max_features="auto")

    # Train the new model
    new_model.fit(new_train_X, new_train_Y)

    # Evaluate the new model
    new_test_pred = new_model.predict(new_test_X)
    new_accuracy = accuracy_score(new_test_Y, new_test_pred)

    return new_accuracy

## Defining the forward selection algorithm 

In [45]:
def forward_selection():
    # Start the timer
    start_time = time.time()

    selected_features = []  # Starting with an empty feature set
    selected_features_accuracy = []  # For storing accuracy of selected feature
    condition = False
    count = 0  # To know how many times the loop runs
    accuracy = -np.inf  # Starting with a very low accuracy

    while not condition:
        remaining_features = [feature for feature in total_features_list if feature not in selected_features]

        temp_accuracy_store = []
        temp_feature_array = []

        for feature in remaining_features:
            # Add one feature to the feature set
            new_features = selected_features + [feature]
            new_accuracy = training_data(new_features)
            temp_accuracy_store.append(new_accuracy)
            temp_feature_array.append(feature)

        maximum_accuracy = max(temp_accuracy_store)

        # Check for convergence. Stop when no improvement.
        if maximum_accuracy <= accuracy:
            condition = True
            break

        # Update the baseline feature set if the accuracy score improved
        else:
            max_idx = np.argmax(temp_accuracy_store)
            accuracy = maximum_accuracy  # Update the accuracy
            selected_features.append(temp_feature_array[max_idx])  # Update the selected features
            selected_features_accuracy.append(accuracy)
            count += 1

    # Stop the timer and calculate the elapsed time
    end_time = time.time()
    training_time = end_time - start_time
    return selected_features, selected_features_accuracy, training_time

In [46]:
selected_features, selected_features_accuracy, training_time = forward_selection()

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

...

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove thi

In [106]:
selected_features

['MOLDWIDTHTOPMEAN',
 'TUNDISHTEMPERATUREMIN',
 'DAYSAFTERSPRAYCLEANINGMEAN',
 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN']

In [107]:
selected_features[-1]

'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN'

In [108]:
selected_features_accuracy[-1]

0.8888888888888888

## Adding again features to selected features to see if accuracy is around 81% for any other variable too.

In [109]:
def forward_selection_again():
    # Start the timer
    start_time = time.time()
     
    remaining_features = [feature for feature in total_features_list if feature not in selected_features]
    
    selected_features_copy=selected_features.copy()
    selected_features_accuracy_copy= selected_features_accuracy.copy()

    for feature in remaining_features:
        # Add one feature to the feature set
        new_features = selected_features + [feature]
        print(new_features)
        new_accuracy = training_data(new_features)
        if 0.83 < new_accuracy <=selected_features_accuracy[-1]:
            selected_features_copy.append(feature)
            selected_features_accuracy_copy.append(new_accuracy)

    # Stop the timer and calculate the elapsed time
    end_time = time.time()
    training_time = end_time - start_time
    return selected_features_copy, selected_features_accuracy_copy, training_time

In [110]:
selected_features_copy, selected_features_accuracy_copy, training_time = forward_selection_again()

['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'SPEEDMEAN']
['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'STOPPERMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'TUNDISHWEIGHTMEAN']
['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'TUNDISHTEMPERATUREMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'TUNDISHTEMPERATUREMAX']
['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'BROADFACE1WATERFLOWMEAN']


...

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'PER_TICMEAN']
['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'PER_MNSMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'PER_VCMEAN']
['MOLDWIDTHTOPMEAN', 'TUNDISHTEMPERATUREMIN', 'DAYSAFTERSPRAYCLEANINGMEAN', 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'PER_NBNMEAN']


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [111]:
len(selected_features_copy)

21

In [112]:
selected_features_copy

['MOLDWIDTHTOPMEAN',
 'TUNDISHTEMPERATUREMIN',
 'DAYSAFTERSPRAYCLEANINGMEAN',
 'SEG1BOTSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'TUNDISHWEIGHTMEAN',
 'BROADFACE2WATERFLOWMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETPRESSMEAN',
 'BROADFACE1HEATFLUXMEAN',
 'BROADFACE1HEATFLUXMIN',
 'SEG0ASECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'SEG5+6BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'SEG0ASECONDARYCOOLINGWATERFLOWSETPOINTVARIANCE',
 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWSETPOINTVARIANCE',
 'SEG2TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG5+6BOTTOMSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'STRANDDRIVECURRENTMIN',
 'STRANDDRIVECURRENTMAX',
 'PER_SIMEAN',
 'DAYSAFTERCIRCLARDMEAN',
 'PER_NBCMEAN']

In [113]:
selected_features_accuracy_copy

[0.797979797979798,
 0.8080808080808081,
 0.8585858585858586,
 0.8888888888888888,
 0.8686868686868687,
 0.8484848484848485,
 0.8383838383838383,
 0.8484848484848485,
 0.8383838383838383,
 0.8383838383838383,
 0.8484848484848485,
 0.8585858585858586,
 0.8383838383838383,
 0.8585858585858586,
 0.8383838383838383,
 0.8383838383838383,
 0.8888888888888888,
 0.8383838383838383,
 0.8484848484848485,
 0.8383838383838383,
 0.8383838383838383]